# File handling

```python
open("filename", mode) # mode(r, a, w, x, t, b) could be to read, write, update
```

- "r" - read: Default value. Opens a file for reading, it returns an error if the files does not exist.
- "a" - append: Opens a file for appending to the end of the file, creates the file if it does not exist.
- "w" - write: Opens a file for writing and overwriting any existing content, creates the file if it does not exist.
- "t" - text: Default value. Text mode
- "b" - binary: Binary mode (e.g. images)

```python
f = open("files/example.txt") # default is read
print(f) # <_io.TextIOWrapper name='./files/reading_file_example.txt' mode='r' encoding='UTF-8'>
```


### Reading methods:


- read(): read the whole text as string. read(number) to limit `the number of characters` we want to read.


In [ ]:
from io import TextIOWrapper

f: TextIOWrapper = open("files/example.txt")
text: str = f.read()
print(type(text), "\n")
print(text)
f.close()

<class 'str'> 

Hello, this is a sample text file for testing purposes.

Name: John Doe
Age: 28
Email: john.doe@example.com

Favorite Programming Languages:
- Python
- Java
- C++

Quote of the day:
"Code is like humor. When you have to explain it, it’s bad." — Cory House



In [ ]:
# Instead of printing all the text, we will limit the numbers of characters

from io import TextIOWrapper

f: TextIOWrapper = open("files/example.txt")
text: str = f.read(29)
print(text)

Hello, this is a sample text 


- readline(): read only one the first line


In [ ]:
from io import TextIOWrapper

f: TextIOWrapper = open("files/example.txt")
text: str = f.readline()
print(text)

Hello, this is a sample text file for testing purposes.



- readlines(): read all the text line by line and returns a list of lines


In [ ]:
from io import TextIOWrapper

f: TextIOWrapper = open("files/example.txt")
texts: list[str] = f.readlines()
print(texts)

['Hello, this is a sample text file for testing purposes.\n', '\n', 'Name: John Doe\n', 'Age: 28\n', 'Email: john.doe@example.com\n', '\n', 'Favorite Programming Languages:\n', '- Python\n', '- Java\n', '- C++\n', '\n', 'Quote of the day:\n', '"Code is like humor. When you have to explain it, it’s bad." — Cory House\n']


- .read().splitlines(): also get all the lines as a list


In [ ]:
from io import TextIOWrapper

f: TextIOWrapper = open("files/example.txt")
texts: list[str] = f.read().splitlines()
print(texts)

['Hello, this is a sample text file for testing purposes.', '', 'Name: John Doe', 'Age: 28', 'Email: john.doe@example.com', '', 'Favorite Programming Languages:', '- Python', '- Java', '- C++', '', 'Quote of the day:', '"Code is like humor. When you have to explain it, it’s bad." — Cory House']


- After we open a file, we should close it


In [ ]:
from io import TextIOWrapper

f: TextIOWrapper = open("files/example.txt")
f.close()

- But there is a high tendency of forgetting to close them.
- There is new way of opening files using `with` - closes the files by itself


In [ ]:
with open("files/example.txt") as f:
    lines: list[str] = f.read().splitlines()
    print(type(lines))
    print(lines)

<class 'list'>
['Hello, this is a sample text file for testing purposes.', '', 'Name: John Doe', 'Age: 28', 'Email: john.doe@example.com', '', 'Favorite Programming Languages:', '- Python', '- Java', '- C++', '', 'Quote of the day:', '"Code is like humor. When you have to explain it, it’s bad." — Cory House']


##### Opening files for Writing and Updating

- f.write() returns the number of characters that have wrote
- if you want to write only and do not care the returned value, assign value to `_` (pass intentionally)
- `_` also use in another case, not just in this lesson.


In [ ]:
with open("files/example.txt", "a") as f:
    _ = f.write("This text has to be appended at the end")

In [ ]:
with open("files/writing_example.txt", "w") as f:
    _ = f.write("This text will be written")

### The `with` Statement

The `with` keyword in Python is used for **automatic resource management**.

Specifically, it ensures that:

1.  The resource is **opened** at the beginning of the code block.
2.  The resource is **automatically released (closed)** when the block finishes, **even if an error occurs** halfway through.

---

#### Without `with` (Bad Practice)

```python
f = open("data.txt", "r")
content = f.read()
f.close()
```

If an error accidentally occurs before $\text{f.close()}$ is called, the file will not be closed, leading to a **resource leak**.

#### Using `with` (Good Practice)

```python
with open("data.txt", "r") as f:
    content = f.read()
    print(content)
```

When exiting the $\text{with}$ block (even if an error occurs), Python automatically calls $\text{f.close()}$.

---

### Context Management Protocol

When leave the block `with` (even error), Python automatically call `f.close()`. This mechanism is called the **Context Management Protocol** and involves two special methods:

| Method        | Purpose                                                            |
| :------------ | :----------------------------------------------------------------- |
| `__enter()__` | Runs when the $\text{with}$ block starts.                          |
| `__exit()__`  | Runs when the $\text{with}$ block is exited (including on errors). |

#### Example: Mock Connection Management

```python
class MyConnection:
    def __enter__(self):
        print("Connection is opened")
        return self # The object returned is bound to 'as conn'

    def __exit__(self, exc_type, exc_val, exc_tb):
        # exc_type, exc_val, exc_tb hold error information if an exception occurred
        print("Connection is closed (regardless of errors)")
        # If this method returns False, the exception is re-raised. (Default behavior)

    def send(self, data):
        print(f"Sending data: {data}")

# Test usage:
with MyConnection() as conn:
    conn.send("Hello Server")
    # raise ValueError("Simulated error") # Try uncommenting to see __exit__ still run

# Output:
# Connection is opened
# Sending data: Hello Server
# Connection is closed (regardless of errors)
```

---

### Error Handling with `__exit()__`

The `__exit()__` method can influence how exceptions are handled:

- **Return $\text{False}$** (or any falsy value): The error will be **re-raised** (propagated) outside the $\text{with}$ block (this is the default behavior).
- **Return $\text{True}$**: The error will be **suppressed** (caught) and will not be propagated outside the $\text{with}$ block, allowing the program to continue.

#### Example: Safe Block

```python
class SafeBlock:
    def __enter__(self):
        print("Starting safe block")

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("An error occurred:", exc_val)
        return True # Suppress the exception (don't re-raise it)

with SafeBlock():
    1 / 0 # division by zero error
    print("This line won't be reached because of the error") # This line is skipped

print("The program continues to run")

# Output:
# Starting safe block
# An error occurred: division by zero
# The program continues to run
```


### Deleting files


In [ ]:
# create deleting files
import os
from time import sleep

with open("files/deleting_files.txt", "w") as f:
    _ = f.write("")

sleep(5)  # pauses 5 seconds to see what will happen

# delete
os.remove("files/deleting_files.txt")

In [ ]:
# If the file does not exist, the remove method will raise an error, so it is good to use a condition like this

import os

if os.path.exists("files/deleting_files.txt"):
    os.remove("files/deleting_files.txt")
else:
    print("The file does not exist")

The file does not exist


### File types

- .txt
- .json (JavaScript Object Notation)
- .csv (Comma Separated values)
- .xlsx (excel file)
- .xml (check XML docs: <a href="https://docs.python.org/2/library/xml.etree.elementtree.html">documentation</a>)


- json.load(): reading json file
- json.loads():changing json to dictionary
- json.dump(): writing json a file
- json.dumps(): changing dictionary to json

So basically, json method without 's' use to read/write file, json method with 's' use to work with dictionary.


In [ ]:
# changing json to dictionary (using json module and loads(var_json) method)

import json

# json is basically a string that contains the data
# so we have to convert it into Python's internal dictionary type
# (even though they look very similar)
person_json: str = """{
    "name": "Apple",
    "country": "Finland",
    "city": "Vietnam",
    "skills": ["JS", "React", "Python"]
}"""

person_dict: dict[str, str | list[str]] = json.loads(person_json)

print(type(person_dict))
print(person_dict)
print(person_dict["name"])

<class 'dict'>
{'name': 'Apple', 'country': 'Finland', 'city': 'Vietnam', 'skills': ['JS', 'React', 'Python']}
Apple


In [ ]:
# changing dictionary to json
# (using dumps(var_dict) method from json module)

import json

# python dictionary
person: dict[str, str | list[str]] = {
    "name": "Asabeneh",
    "country": "Finland",
    "city": "Helsinki",
    "skills": ["JavaScrip", "React", "Python"],
}

person_json = json.dumps(
    person, indent=4
)  # indent could be 2, 4, 8. It beautifies the json
print(type(person_json))
print(person_json)

<class 'str'>
{
    "name": "Asabeneh",
    "country": "Finland",
    "city": "Helsinki",
    "skills": [
        "JavaScrip",
        "React",
        "Python"
    ]
}


In [ ]:
# Saving as JSON file
# writing json a file, use json.dump() method (different than json.dumps() above)
# it can take dictionary, output file, ensure_ascii and indent.

import json

person: dict[str, str | list[str]] = {
    "name": "Apple",
    "country": "Finland",
    "city": "Vietnam",
    "skills": ["JavaScript", "Python"],
}

with open("files/json_example.json", "w", encoding="utf-8") as f:
    json.dump(person, f, ensure_ascii=False, indent=4)